# Tutorial 4: Attaching a Dataset

In [ ]:
import libspn as spn
import tensorflow as tf

### Building a Test Graph with Random Weights

In [ ]:
indicators_x = spn.IndicatorLeaf(num_vars=2, num_vals=2, name="indicators_x")
sum_11 = spn.Sum((indicators_x, [0,1]), name="sum_11")
sum_12 = spn.Sum((indicators_x, [0,1]), name="sum_12")
sum_21 = spn.Sum((indicators_x, [2,3]), name="sum_21")
sum_22 = spn.Sum((indicators_x, [2,3]), name="sum_22")
prod_1 = spn.Product(sum_11, sum_21, name="prod_1")
prod_2 = spn.Product(sum_11, sum_22, name="prod_2")
prod_3 = spn.Product(sum_12, sum_22, name="prod_3")
root = spn.Sum(prod_1, prod_2, prod_3, name="root")
latent_indicator_y = root.generate_latent_indicators(name="latent_indicator_y")
spn.generate_weights(root, initializer=tf.initializers.random_uniform(0, 1))

### Visualizing the SPN Graph

In [ ]:
spn.display_spn_graph(root)

### Specify Training Data

In [ ]:
dataset=spn.CSVFileDataset('data.csv', num_vals=[2, 2], defaults=[[-1],[-1],[-1]],
                           num_labels=1, num_epochs=10, batch_size=10, shuffle=False)
samples, labels = dataset.get_data()
indicators_x.attach_feed(samples)
latent_indicator_y.attach_feed(labels)

### Add Learning Ops

In [ ]:
init_weights = spn.initialize_weights(root)
learning = spn.HardEMLearning(root, initial_accum_value=2)
init_learning = learning.reset_accumulators()
accumulate_updates = learning.accumulate_updates()
update_spn = learning.update_spn()
likelihood = tf.reduce_mean(learning.value.values[root])

### Run Learning

In [ ]:
epoch = 0
with spn.session() as (sess, run):
    sess.run(init_weights)
    sess.run(init_learning)
    try:    
        while run():
            likelihood_arr, _ = sess.run([likelihood, accumulate_updates])
            print("Avg. Likelihood: %s" % (likelihood_arr))
            sess.run(update_spn)
            
    except tf.errors.OutOfRangeError:
        print("Done!")

In [ ]:
with spn.session() as (sess, run):
    sess.run(init_weights)
    sess.run(init_learning)
    try:
        while run():
            likelihoods, _ =  sess.run([likelihood, accumulate_updates])
            print("Avg. Likelihood: %s" % (avg_likelihood))
            sess.run(update_spn)
            
    except tf.errors.OutOfRangeError:
        print("TRAINING DONE!")